# Initial Model Testing with new Sequence Generator

In [23]:
import numpy as np
import pandas as pd 

#Keras
from tensorflow import keras
from tensorflow import stack, Tensor
from keras import Sequential, layers
from keras.callbacks import EarlyStopping
from keras.optimizers import RMSprop
from keras.callbacks import LearningRateScheduler
from keras.utils import timeseries_dataset_from_array
from tensorflow import data



#Project imports 
from bdi_predict.model.data import clean_data
from bdi_predict.model.preprocessor import train_val_test_split, min_max_scaler
from bdi_predict.model.sequencer import SequenceGenerator
from bdi_predict.model.params import BASE_PROJECT_PATH


In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Preprocessing - (splitting data, scaling, and creating sequences)

In [25]:
df = pd.read_csv("../data/cleaned_data.csv")
df = df.drop(columns="log_BDRY").copy()
df.set_index("time", inplace=True)
df.head(3)

,BDRY,BDI,target
time,,,
2018-03-23,24.1005,1122.0,-0.024735
2018-03-26,24.8400,1126.0,0.013126
2018-03-27,24.0800,1117.0,-0.013495


In [26]:
dfs = train_val_test_split(df=df, train_val_test_ratio=(7,2,1))

Data split into train, validation, and test datasets.


In [27]:
df_train, df_val, df_test = min_max_scaler(dfs=dfs)

Datasets min-max scaled.


In [28]:
df_train.head(3)

,BDRY,BDI,target
0,0.769748,0.240992,0.337520
1,0.797941,0.242314,0.531249
2,0.768967,0.239339,0.395035


In [29]:
df_val.head(3)

,BDRY,BDI,target
0,0.865040,0.984463,0.453253
1,0.858178,0.984463,0.448999
2,0.880671,0.985785,0.513167


In [30]:
df_test.head(3)

,BDRY,BDI,target
0,0.462066,0.598678,0.417447
1,0.488753,0.592727,0.559068
2,0.505528,0.610579,0.521776


In [31]:
#Instantiating a SequenceGenerator Class

Sequencer = SequenceGenerator(input_width=20,
                                      target_width=1,
                                      offset=1,
                                      df_train=df_train,
                                      df_val=df_val,
                                     df_test=df_test,
                                     target_columns=["target"])

Sequencer

Total sequence size: 21
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
Target indice(s): [20]
Target column name(s): ['target']

In [32]:
#Understand how exactly the split_sequence works by understading what exactly I need to feed in and in what manner
#for the LSTM model!

In [34]:
# Inputting a tf.Tensor made up of 8 slices 
#(the 8 seuqnecs that make up a single batch of batch_size=8) to generate

# Stack 8 slices, the length of the total window.
# FIX THIS!
example_sequence = stack([np.array(df_train[:Sequencer.total_sequence_size]),
                           np.array(df_train[100:100+Sequencer.total_sequence_size]),
                           np.array(df_train[200:200+Sequencer.total_sequence_size])])

example_inputs, example_targets = Sequencer.split_sequence(example_sequence)

print('All shapes are: (batch, time, features)')
print(f'Sequence shape: {example_sequence.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'targets shape: {example_targets.shape}')

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got slice(20, None, None)

In [15]:
inputs, targets = Sequencer.split_sequence(df=Sequencer.df_train)

InvalidIndexError: (slice(None, None, None), slice(0, 20, None), slice(None, None, None))

In [83]:
SequenceGenerator.input_slice

slice(8, 20, None)

In [15]:
def init_model():
    
    """ 
    Initialize the LSTM Reucrrent Neural Network.
    """
    
    print("\nInitialising model...")
    
    model = Sequential()

    #LSTM LAYERS:
    
    model.add(layers.LSTM(60,
                          activation="tanh",
                          input_shape=(20,2),
                          return_sequences=False))

    #DENSE LAYERS:
    
    model.add(layers.Dense(25, activation="relu"))
    model.add(layers.Dense(1, activation="linear"))
    
    print("\nmodel initialized.")

    #SETTING UP OPTIMIZERS:
    
    lr_schedule = ExponentialDecay(initial_learning_rate=1e-3,
                                   decay_steps=10000,
                                   decay_rate=0.9)
    
    rmsprop = RMSprop(learning_rate=lr_schedule)
    
    #COMPILING MODEL:
    
    model.compile(loss="mse",
                  optimizer=rmsprop,
                  metrics="mae")
    print("\nmodel compiled.")
    

    return model

In [23]:
def train_model(model:keras.Sequential,
                XandY:data.Dataset,
                patience=10,
                validation_data=data.Dataset):
    
    """
    Fit model and return a the tuple (fitted_model, history)
    """
    
    print("\nTraining model...")
    
    #EarlyStopping DEFINITION:
    
    es = EarlyStopping(monitor="val_mae",
                       patience=patience,
                       restore_best_weights=True)
    
    #FITTING MODEL:
    
    history = model.fit(XandY,
                        epochs=100,
                        validation_data=validation_data,
                        shuffle=True,
                        callbacks=es)
    
    
    print(f"\nmodel trained ({len(XandY)} rows).")
     
    return model, history

In [24]:
df = pd.read_csv("../data/cleaned_data.csv")